In [1]:
import polars as pl
import requests
import shapely

In [2]:
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTABC.json?key=OBANYC"
).json()
df_oba = pl.from_dicts(r["data"]["list"])
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTA NYCT.json?key=OBANYC"
).json()
# combine the two dataframes
df_oba = pl.concat([df_oba, pl.from_dicts(r["data"]["list"])])

In [3]:
r = requests.get(
    "https://bustime.mta.info/api/siri/vehicle-monitoring.json?key=OBANYC&version=2"
).json()
parsed = [
    v["MonitoredVehicleJourney"]
    for v in r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0][
        "VehicleActivity"
    ]
]
df_siri = pl.from_dicts(
    parsed
    # r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0]["VehicleActivity"]
)

In [4]:
df_siri = df_siri.with_columns(
    pl.col("VehicleRef").str.split("_").list.last().alias("vehicle_id")
)
df_oba = df_oba.with_columns(
    pl.col("vehicleId").str.split("_").list.last().alias("vehicle_id")
)
# [
#     "VehicleID"
# ]
# create polars series with vehicle IDs

In [5]:
display(df_oba, df_siri)

lastLocationUpdateTime,lastUpdateTime,location,occupancyCapacity,occupancyCount,occupancyStatus,phase,status,tripId,vehicleId,vehicle_id
i64,i64,struct[2],i64,i64,i64,str,str,str,str,str
1748650302254,1748650302254,"{40.629971,-73.944794}",80,10,1,"""deadhead_during""","""blockInf""","""MTABC_43012506-FRPB5-FR_B5-Wee…","""MTABC_9128""","""9128"""
1748650313091,1748650313091,"{40.661297,-73.774529}",null,null,null,"""at_base""","""default""","""MTABC_42507761-JKPB5-JK_B5-Wee…","""MTABC_9129""","""9129"""
1748650316000,1748650316000,"{40.704886,-73.809272}",null,null,null,"""layover_during""","""blockInf""","""MTABC_42505869-JKPB5-JK_B5-Wee…","""MTABC_9126""","""9126"""
1748650300000,1748650300000,"{40.702018,-73.809872}",80,6,1,"""in_progress""","""blockInf""","""MTABC_42507765-JKPB5-JK_B5-Wee…","""MTABC_9127""","""9127"""
1748650107253,1748650107253,"{40.660549,-73.774239}",null,null,null,"""at_base""","""default""","""MTABC_42505626-JKPB5-JK_B5-Wee…","""MTABC_9124""","""9124"""
…,…,…,…,…,…,…,…,…,…,…
1748650321000,1748650321000,"{40.625087,-74.148472}",65,4,1,"""in_progress""","""default""","""MTA NYCT_CA_B5-Weekday-SDon-11…","""MTA NYCT_2545""","""2545"""
1748648713000,1748648713000,"{40.574184,-74.165695}",null,null,null,"""at_base""","""default""","""""","""MTA NYCT_2546""","""2546"""
1748648578000,1748648578000,"{40.574532,-74.165611}",null,null,null,"""at_base""","""default""","""""","""MTA NYCT_2547""","""2547"""


LineRef,DirectionRef,FramedVehicleJourneyRef,PublishedLineName,DestinationRef,DestinationName,Monitored,VehicleLocation,Bearing,ProgressRate,Occupancy,VehicleRef,MonitoredCall,SituationRef,OriginAimedDepartureTime,ProgressStatus,vehicle_id
str,str,struct[2],list[str],str,list[str],bool,struct[2],f64,str,str,str,struct[9],list[struct[1]],str,list[str],str
"""MTA NYCT_BX28""","""1""","{""2025-05-30"",""MTA NYCT_GH_B5-Weekday-SDon-118500_BX238_633""}","[""Bx28""]","""MTA_101530""","[""FORDHAM CENTER 192 ST via GUNHILL""]",true,"{-73.869681,40.878185}",164.57784,"""normalProgress""","""seatsAvailable""","""MTA NYCT_9771""","{""2025-05-30T20:17:16.707-04:00"",""2025-05-30T20:13:41.978-04:00"",""< 1 stop away"",""2025-05-30T20:13:41.978-04:00"",320,0,{{10,80,""manySeatsAvailable""}},1,[""EAST GUN HILL RD/HULL AV""]}",null,null,null,"""9771"""
"""MTA NYCT_Q17""","""0""","{""2025-05-30"",""MTA NYCT_JA_B5-Weekday-SDon-120700_MISC_854""}","[""Q17""]","""MTA_804041""","[""FLUSHING""]",true,"{-73.793887,40.708477}",123.19985,"""normalProgress""",null,"""MTA NYCT_8440""","{""2025-05-30T20:09:59.713-04:00"",""2025-05-30T20:12:11.232-04:00"",""< 1 stop away"",""2025-05-30T20:12:11.232-04:00"",154,0,null,1,[""168 ST/HILLSIDE AV""]}","[{""MTABC_lmm:planned_work:23593""}]",null,null,"""8440"""
"""MTA NYCT_BX29""","""0""","{""2025-05-30"",""MTA NYCT_GH_B5-Weekday-SDon-119500_BX12_164""}","[""Bx29""]","""MTA_100829""","[""CITY ISLAND""]",true,"{-73.785252,40.843927}",290.83667,"""normalProgress""","""seatsAvailable""","""MTA NYCT_9770""","{""2025-05-30T20:05:10.719-04:00"",""2025-05-30T20:11:43.520-04:00"",""at stop"",""2025-05-30T20:11:43.520-04:00"",10,0,{{7,80,""manySeatsAvailable""}},1,[""CITY ISLAND AV/WINTERS ST""]}",null,null,null,"""9770"""
"""MTA NYCT_Q84""","""1""","{""2025-05-30"",""MTA NYCT_JA_B5-Weekday-SDon-119500_MISC_858""}","[""Q84""]","""MTA_804039""","[""JAMAICA PARSONS-ARCHER""]",true,"{-73.761007,40.686489}",199.52666,"""normalProgress""","""seatsAvailable""","""MTA NYCT_8443""","{""2025-05-30T20:06:50.727-04:00"",null,""at stop"",null,0,0,{{14,80,""manySeatsAvailable""}},1,[""120 AV/FARMERS BLVD""]}","[{""MTA NYCT_lmm:planned_work:18177""}, {""MTABC_lmm:planned_work:23593""}]",null,null,"""8443"""
"""MTA NYCT_M4""","""0""","{""2025-05-30"",""MTA NYCT_MV_B5-Weekday-SDon-118500_M4_436""}","[""M4""]","""MTA_400636""","[""WSHNGTN HTS CABRINI via MADISON via BWAY""]",true,"{-73.961868,40.776666}",53.753784,"""normalProgress""","""seatsAvailable""","""MTA NYCT_9775""","{""2025-05-30T20:08:12.737-04:00"",null,""at stop"",null,0,0,{{12,80,""manySeatsAvailable""}},1,[""MADISON AV/E 79 ST""]}","[{""MTA NYCT_lmm:planned_work:24031""}]",null,null,"""9775"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MTABC_Q10""","""1""","{""2025-05-30"",""MTABC_42506931-JKPB5-JK_B5-Weekday-02-SDon""}","[""Q10""]","""MTA_982006""","[""JFK AIRPORT via LEFFERTS BL via 130 ST""]",true,"{-73.811481,40.66103}",168.97156,"""normalProgress""",null,"""MTABC_5376""","{""2025-05-30T20:13:00.092-04:00"",""2025-05-30T20:14:35.852-04:00"",""0.9 miles away"",""2025-05-30T20:14:35.852-04:00"",1395,0,null,1,[""LEFFERTS BLVD/AIRTRAIN STATION""]}","[{""MTABC_lmm:planned_work:23593""}]",null,null,"""5376"""
"""MTABC_Q52+""","""1""","{""2025-05-30"",""MTABC_42508270-JKPB5-JK_B5-Weekday-02-SDon""}","[""Q52-SBS""]","""MTA_553336""","[""+SELECT BUS ARVERNE B. 54 ST via WOODHAVEN BL via CROSS BAY BL""]",true,"{-73.840088,40.659566}",284.24493,"""normalProgress""","""seatsAvailable""","""MTABC_5378""","{""2025-05-30T20:10:13.096-04:00"",""2025-05-30T20:14:04.720-04:00"",""< 1 stop away"",""2025-05-30T20:14:04.720-04:00"",783,0,{{8,100,""manySeatsAvailable""}},1,[""CROSS BAY BLVD/163 AV""]}","[{""MTABC_lmm:planned_work:16452""}, {""MTABC_lmm:planned_work:23593""}]",null,null,"""5378"""
"""MTABC_Q10""","""1""","{""2025-05-30"",""MTABC_42506820-JKPB5-JK_B5-Weekday-02-SDon""}","[""Q10""]","""MTA_550290""","[""150 AV via LEFFERTS BL via 130 ST""]",true,"{-73.829534,40.713627}",336.4589,"""

In [6]:
siri_vehicle_ids = df_siri["vehicle_id"]
oba_vehicle_ids = df_oba["vehicle_id"]


display(
    "OBA difference:", oba_vehicle_ids.filter(~oba_vehicle_ids.is_in(siri_vehicle_ids))
)
# display("difference:", siri_vehicle_ids.filter(~siri_vehicle_ids.is_in(oba_vehicle_ids)))

'OBA difference:'

vehicle_id
str
"""9128"""
"""9129"""
"""9124"""
"""9125"""
"""9123"""
…
"""2544"""
"""2546"""
"""2547"""


In [7]:
filtered_df = df_oba.filter(
    pl.col("lastLocationUpdateTime") != pl.col("lastUpdateTime")
)
# rows are equal
# print(f"Total rows: {len(df)}")
# print(f"Filtered rows: {len(filtered_df)}")
# filtered_df.head(5)
filtered_df

lastLocationUpdateTime,lastUpdateTime,location,occupancyCapacity,occupancyCount,occupancyStatus,phase,status,tripId,vehicleId,vehicle_id
i64,i64,struct[2],i64,i64,i64,str,str,str,str,str


In [8]:
siri_diff = siri_vehicle_ids.filter(~siri_vehicle_ids.is_in(oba_vehicle_ids))
print(f"Siri vehicle IDs not in OBA: {siri_diff}")

Siri vehicle IDs not in OBA: shape: (0,)
Series: 'vehicle_id' [str]
[
]


In [9]:
df_oba["phase"].value_counts()

phase,count
str,u32
"""layover_before""",3
"""in_progress""",1912
"""deadhead_during""",431
"""at_base""",2015
"""layover_during""",281
"""spooking""",19
"""deadhead_before""",557


In [ ]:
df_siri["ProgressRate"].value_counts()

ProgressRate,count
str,u32
"""normalProgress""",3203
"""noProgress""",318
"""unknown""",1


In [ ]:
df_siri["ProgressStatus"].value_counts()

ProgressStatus,count
list[str],u32
"[""layover""]",309
null,3212
"[""spooking""]",1
